In [1]:
!pip3 install --upgrade pip
!pip3 install Kqlmagic --no-cache-dir --upgrade
!pip3 install nbformat

    100% |████████████████████████████████| 1.6MB 628kB/s 
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Keyring is skipped due to an exception: org.freedesktop.DBus.Error.InvalidFileContent: D-Bus library appears to be incorrectly set up: see the manual page for dbus-uuidgen to correct this issue. (Failed to open "/var/lib/dbus/machine-id": No such file or directory; UUID file '/etc/machine-id' should contain a hex string of length 32, not length 0, with no other text)
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 229 kB 8.5 MB/s 
     |████████████████████████████████| 97 kB 17.2 MB/s 
     |████████████████████████████████| 291 kB 15.8 MB/s 
     |████████████████████████████████| 11.5 MB 3.8 MB/s 
     |████████████████████████████████| 9.5 MB 82.6 MB/s 
     |██████████████

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px

Add Kqlmagic to notebook magics

In [ ]:
%reload_ext Kqlmagic

In [ ]:
%kql loganalytics://code;workspace='1054c1b0-f063-40a7-823e-91dac02fbabc';alias='arturol76-edge-benchmark'

```bash
cd ~/dev-linux/edge-stress-tests/edge-benchmark/scripts/
```

start the VM:
```bash
az vm start -n standard-ds3-v2-edge-1-2-1626596809 -g edge-benchmark-vm5-rg
```

create the VM and provisions IoT Edge:
```bash
./create.sh -s Standard_DS3_v2 -g edge-benchmark-vm5-rg -d ../tests/empty-1.2.json -e 1.2
```

login
```bash
ssh arturo@standard-ds3-v2-edge-1-2-1626596809.westeurope.cloudapp.azure.com -i ../keys/vm-key
```

deploy the test
```bash
az iot edge set-modules -n arturol76-s1-benchmark -d standard-ds3-v2-edge-1-2-1626596809 --content ../tests/test30-low-rate-metrics-1.2.json
```

In [ ]:
from datetime import datetime, timedelta
from dateutil import parser
testStart=parser.parse("2021-07-29T08:02:00Z")
testStop=parser.parse("2021-07-29T08:04:00Z")

In [ ]:
%%kql
    let _start=testStart;
    let _stop=testStop;
    InsightsMetrics 
    | where Origin == "iot.azm.ms" and Namespace == "metricsmodule" 
    | where Name == "edgehub_queue_length" 
    | extend dimensions=parse_json(Tags) 
    | extend device = tostring(dimensions.edge_device) 
    | extend ep = tostring(dimensions.endpoint) 
    | where  ep == "iothub"
    | extend qlen = toint(Val)
    | where qlen > 0 and qlen != 2490317 
    | where TimeGenerated > _start and TimeGenerated < _stop 
    | project TimeGenerated, qlen
    | render timechart title='edgehub_queue_length'

In [ ]:
df = _kql_raw_result_.to_dataframe()
df = df.sort_values(by=['TimeGenerated'])

In [ ]:
qlenDf = df

In [ ]:
qlenDf

In [ ]:
%%kql
    let _start=testStart;
    let _stop=testStop;
    let _module_id="source";
    InsightsMetrics
    | where Origin == "iot.azm.ms" and Namespace == "metricsmodule"
    | where Name == "rate"
    | extend dimensions = parse_json(Tags)
    | extend module_id = dimensions.module_id
    | extend channel = dimensions.input_index
    | where module_id == _module_id
    | where TimeGenerated > _start and TimeGenerated < _stop
    | project TimeGenerated, Val

In [ ]:
df = _kql_raw_result_.to_dataframe()
df = df.sort_values(by=['TimeGenerated'])

In [ ]:
sourceRateDf = df

In [ ]:
%%kql
    let _start=testStart;
    let _stop=testStop;
    let _module_id="sink";
    let _input_index="input1";
    InsightsMetrics
    | where Origin == "iot.azm.ms" and Namespace == "metricsmodule"
    | where Name == "rate"
    | extend dimensions = parse_json(Tags)
    | extend module_id = dimensions.module_id
    | extend channel = dimensions.input_index
    | where module_id == _module_id and dimensions.input_index == _input_index
    | where TimeGenerated > _start and TimeGenerated < _stop
    | project TimeGenerated, Val

In [ ]:
df = _kql_raw_result_.to_dataframe()
df = df.sort_values(by=['TimeGenerated'])

In [ ]:
sinkRateDf = df

In [ ]:
sinkRateDf

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)

fig.add_trace( 
    go.Line(x=sourceRateDf['TimeGenerated'], y=sourceRateDf['Val'], name='source'), row=3, col=1
)

fig.add_trace( 
    go.Line(x=sinkRateDf['TimeGenerated'], y=sinkRateDf['Val'], name='sink'), row=2, col=1
)

fig.add_trace( 
    go.Line(x=qlenDf['TimeGenerated'], y=qlenDf['qlen'], name='qlen'), row=1, col=1
)


fig.update_layout(title_text="Results")
fig.show()

In [ ]:
data = {'Source':  [sourceRateDf['Val'].mean(), sourceRateDf['Val'].min(), sourceRateDf['Val'].max()],
        'Sink': [sinkRateDf['Val'].mean(), sinkRateDf['Val'].min(), sinkRateDf['Val'].max()]
        }

df = pd.DataFrame (data, columns = ['Source','Sink'], index=['avg', 'min', 'max'])

print (df)